In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import albumentations as A
import numpy as np
from tqdm import tqdm

# Define paths
IMG_DIR = "/content/drive/MyDrive/yolo_dataset/images-orignal/train"       # Input images
LABEL_DIR = "/content/drive/MyDrive/yolo_dataset/labels-orignal/train"     # Input labels (.txt)
OUT_IMG_DIR = "/content/drive/MyDrive/yolo_dataset1/images/train"          # Output images
OUT_LABEL_DIR = "/content/drive/MyDrive/yolo_dataset1/labels/train"        # Output labels

# Make output directories
os.makedirs(OUT_IMG_DIR, exist_ok=True)
os.makedirs(OUT_LABEL_DIR, exist_ok=True)

# Albumentations augmentations (each wrapped in A.Compose)
AUGS = {
    "flip": A.Compose([
        A.HorizontalFlip(p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),

    "rotate_plus": A.Compose([
        A.Rotate(limit=(5, 5), p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),

    "rotate_minus": A.Compose([
        A.Rotate(limit=(-5, -5), p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),

    "scale": A.Compose([
        A.Affine(scale=(0.8, 0.8), p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))
}

# Read all image files
img_files = [f for f in os.listdir(IMG_DIR) if f.endswith((".jpg", ".png"))]

# Helper to read YOLO labels
def read_yolo_labels(label_path, img_w, img_h):
    bboxes = []
    with open(label_path, 'r') as f:
        for line in f:
            cls, x, y, w, h = map(float, line.strip().split())
            x1 = (x - w / 2) * img_w
            y1 = (y - h / 2) * img_h
            x2 = (x + w / 2) * img_w
            y2 = (y + h / 2) * img_h
            bboxes.append([x1, y1, x2, y2, int(cls)])
    return bboxes

# Helper to save YOLO labels
def save_yolo_labels(label_path, bboxes, img_w, img_h):
    with open(label_path, 'w') as f:
        for x1, y1, x2, y2, cls in bboxes:
            cx = ((x1 + x2) / 2) / img_w
            cy = ((y1 + y2) / 2) / img_h
            w = (x2 - x1) / img_w
            h = (y2 - y1) / img_h
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

# Augment each image
for img_file in tqdm(img_files):
    img_path = os.path.join(IMG_DIR, img_file)
    label_path = os.path.join(LABEL_DIR, os.path.splitext(img_file)[0] + ".txt")

    image = cv2.imread(img_path)
    if image is None:
        continue
    height, width = image.shape[:2]
    bboxes = read_yolo_labels(label_path, width, height)

    if not bboxes:
        continue  # Skip images without bounding boxes

    for aug_name, aug in AUGS.items():
        transformed = aug(image=image, bboxes=[b[:4] for b in bboxes], class_labels=[b[4] for b in bboxes])
        aug_image = transformed["image"]
        aug_bboxes = [
            list(map(float, bbox)) + [cls] for bbox, cls in zip(transformed["bboxes"], transformed["class_labels"])
        ]

        # Save image
        aug_img_name = f"{os.path.splitext(img_file)[0]}_{aug_name}.jpg"
        cv2.imwrite(os.path.join(OUT_IMG_DIR, aug_img_name), aug_image)

        # Save label
        aug_label_name = f"{os.path.splitext(img_file)[0]}_{aug_name}.txt"
        save_yolo_labels(os.path.join(OUT_LABEL_DIR, aug_label_name), aug_bboxes, aug_image.shape[1], aug_image.shape[0])


100%|██████████| 720/720 [04:29<00:00,  2.67it/s]


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from ultralytics import YOLO
import yaml

# Load model
model = YOLO("yolov8m.pt")

# Load YAML
with open("/content/drive/MyDrive/yolo_dataset1/hyp.yaml", 'r') as f:
    hyp_dict = yaml.safe_load(f)

# Train
model.train(
    data="/content/drive/MyDrive/yolo_dataset1/tb.yaml",
    epochs=200,
    imgsz=512,
    patience=20,
    workers=4,
    batch=16,
    project='/content/drive/MyDrive/yolo_models/YOLOv8m',
    **hyp_dict
)


Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.0777, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.265, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_dataset1/tb.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00941, lrf=0.0202, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.841, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=

train: Scanning /content/drive/MyDrive/yolo_dataset1/labels/train.cache... 2876 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2876/2876 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.8±2.2 ms, read: 34.3±9.6 MB/s, size: 68.8 KB)


val: Scanning /content/drive/MyDrive/yolo_dataset1/labels/val.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/yolo_models/YOLOv8m/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.00941' and 'momentum=0.841' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.00047), 83 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/yolo_models/YOLOv8m/train3
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         80        122      0.487      0.468      0.392      0.183
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 22, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



42 epochs completed in 0.821 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_models/YOLOv8m/train3/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/yolo_models/YOLOv8m/train3/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/yolo_models/YOLOv8m/train3/weights/best.pt...
Ultralytics 8.3.151 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.82it/s]


                   all         80        122       0.53      0.453      0.441       0.19
    ActiveTuberculosis         68        102      0.683      0.657      0.644      0.303
ObsoletePulmonaryTuberculosis         15         20      0.377       0.25      0.238     0.0774
Speed: 0.1ms preprocess, 9.1ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_models/YOLOv8m/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b444148c950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
from ultralytics import YOLO
import yaml

# Load model
model = YOLO("yolov8m.pt")

# Load YAML
with open("/content/drive/MyDrive/yolo_dataset1/hyp.yaml", 'r') as f:
    hyp_dict = yaml.safe_load(f)

# Train
model.train(
    data="/content/drive/MyDrive/yolo_dataset1/tb.yaml",
    epochs=200,
    imgsz=512,
    patience=20,
    workers=4,
    batch=16,
    project='/content/drive/MyDrive/yolo_models/YOLOv8m',
    **hyp_dict
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 49.7M/49.7M [00:00<00:00, 306MB/s]


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.0777, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.265, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/yolo_dataset1/tb.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00941, lrf=0.0202, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.841, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=

100%|██████████| 755k/755k [00:00<00:00, 17.1MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...

     36/200      4.92G   0.008239     0.2466      1.028         27        512: 100%|██████████| 334/334 [02:01<00:00,  2.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.58it/s]


                   all         80        122      0.401      0.426      0.392      0.172
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 16, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

36 epochs completed in 1.241 hours.
Optimizer stripped from /content/drive/MyDrive/yolo_models/YOLOv8m/train2/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/yolo_models/YOLOv8m/train2/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/yolo_models/YOLOv8m/train2/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.21it/s]


                   all         80        122       0.46      0.463      0.418      0.187
    ActiveTuberculosis         68        102      0.551      0.725      0.622      0.251
ObsoletePulmonaryTuberculosis         15         20      0.368        0.2      0.215      0.124
Speed: 0.3ms preprocess, 7.2ms inference, 0.0ms loss, 6.2ms postprocess per image
Results saved to /content/drive/MyDrive/yolo_models/YOLOv8m/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7dbbebaa8290>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
import ultralytics
print(ultralytics.__version__)

8.3.151


In [ ]:
import ultralytics
print(ultralytics.__file__)

/usr/local/lib/python3.11/dist-packages/ultralytics/__init__.py


In [ ]:
!cp /usr/local/lib/python3.11/dist-packages/ultralytics/yolo/engine/loss.py ./loss.py


cp: cannot stat '/usr/local/lib/python3.11/dist-packages/ultralytics/yolo/engine/loss.py': No such file or directory


In [ ]:
import os
import albumentations as A
import cv2

# Define augmentations (using pascal_voc bbox format)
AUGS = {
    "flip": A.Compose([
        A.HorizontalFlip(p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),

    "rotate_plus": A.Compose([
        A.Rotate(limit=(5, 5), p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),

    "rotate_minus": A.Compose([
        A.Rotate(limit=(-5, -5), p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels'])),

    "scale": A.Compose([
        A.Affine(scale=(0.8, 0.8), p=1.0)
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))
}

# Paths
images_dir = "/content/drive/MyDrive/yolo_dataset1/images/train"   # your images folder
labels_dir = "/content/drive/MyDrive/yolo_dataset1/labels/train"   # your labels folder

def yolo_to_pascal_voc(x_center, y_center, width, height, img_w, img_h):
    # Convert normalized YOLO bbox to Pascal VOC bbox (xmin, ymin, xmax, ymax)
    xmin = (x_center - width / 2) * img_w
    ymin = (y_center - height / 2) * img_h
    xmax = (x_center + width / 2) * img_w
    ymax = (y_center + height / 2) * img_h
    return [xmin, ymin, xmax, ymax]

def pascal_voc_to_yolo(xmin, ymin, xmax, ymax, img_w, img_h):
    # Convert Pascal VOC bbox to normalized YOLO bbox
    x_center = ((xmin + xmax) / 2) / img_w
    y_center = ((ymin + ymax) / 2) / img_h
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h
    return [x_center, y_center, width, height]

def read_label_file(label_path):
    classes = []
    bboxes = []
    with open(label_path, "r") as f:
        for line in f:
            if line.strip() == "":
                continue
            parts = line.strip().split()
            class_id = int(float(parts[0]))

            bbox = list(map(float, parts[1:5]))  # x_center, y_center, w, h normalized
            classes.append(class_id)
            bboxes.append(bbox)
    return classes, bboxes

def write_label_file(label_path, classes, bboxes, img_w, img_h):
    with open(label_path, "w") as f:
        for cls, bbox in zip(classes, bboxes):
            # bbox is Pascal VOC -> convert to YOLO normalized
            x_center, y_center, w, h = pascal_voc_to_yolo(*bbox, img_w, img_h)
            # Clamp values between 0 and 1
            x_center = max(min(x_center, 1), 0)
            y_center = max(min(y_center, 1), 0)
            w = max(min(w, 1), 0)
            h = max(min(h, 1), 0)
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

def augment_image_and_labels(image_path, label_path, aug_name, aug_pipeline):
    # Read image
    img = cv2.imread(image_path)
    img_h, img_w = img.shape[:2]

    # Read labels
    classes, bboxes_yolo = read_label_file(label_path)

    # Convert all bboxes from YOLO to pascal_voc for albumentations
    bboxes_pascal = []
    for bbox in bboxes_yolo:
        bboxes_pascal.append(yolo_to_pascal_voc(*bbox, img_w, img_h))

    # Check if class 1 is present (the class to augment)
    if 1 not in classes:
        print(f"Skipping {image_path}, class 1 not found")
        return

    # Apply augmentation
    augmented = aug_pipeline(image=img, bboxes=bboxes_pascal, class_labels=classes)
    aug_img = augmented['image']
    aug_bboxes = augmented['bboxes']
    aug_classes = augmented['class_labels']

    # Save augmented image and label with prefix 'aug_'
    base_name = os.path.basename(image_path)
    aug_image_path = os.path.join(images_dir, f"aug_{aug_name}_{base_name}")
    cv2.imwrite(aug_image_path, aug_img)

    base_label_name = os.path.basename(label_path)
    aug_label_path = os.path.join(labels_dir, f"aug_{aug_name}_{base_label_name}")

    # Write label file converting bboxes back to YOLO normalized format
    write_label_file(aug_label_path, aug_classes, aug_bboxes, img_w, img_h)
    print(f"Saved augmented: {aug_image_path} and {aug_label_path}")

def main():
    # For every image-label pair, augment if class 1 present
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue
        label_path = os.path.join(labels_dir, label_file)
        image_file = label_file.replace(".txt", ".jpg")  # Adjust extension if needed
        image_path = os.path.join(images_dir, image_file)
        if not os.path.exists(image_path):
            print(f"Image {image_path} not found, skipping.")
            continue

        for aug_name, aug_pipeline in AUGS.items():
            augment_image_and_labels(image_path, label_path, aug_name, aug_pipeline)

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_rotate_minus.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_rotate_minus.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_rotate_minus.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_rotate_minus.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_scale.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_scale.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_scale.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0798_scale.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0827_flip.jpg, class 1 not found
Skipping /content/drive/MyDrive/yolo_dataset1/images/train/tb0827_f

In [ ]:
import os
from glob import glob

# Label directories
label_dirs = [
  "/content/drive/MyDrive/yolo_dataset1/labels/train"
    # "/content/drive/MyDrive/yolo_dataset1/labels/val"
    # "/content/dataset/labels/test"  # add if needed
]

# Class count dictionary (assuming two classes)
class_counts = {0: 0, 1: 0}

for label_dir in label_dirs:
    txt_files = glob(os.path.join(label_dir, "*.txt"))
    for file in txt_files:
        with open(file, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                try:
                    class_id = int(float(line.split()[0]))  # Handles '0.0' or '1.0'
                    if class_id in class_counts:
                        class_counts[class_id] += 1
                    else:
                        class_counts[class_id] = 1
                except ValueError:
                    print(f"Skipping invalid line in {file}: {line.strip()}")

# Print results
print("Class Sample Counts:")
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} instances")


Class Sample Counts:
Class 0: 3960 instances
Class 1: 4380 instances
